# Pytorch Distributed

## 设置机器上可用的 CUDA 设备

我们的 GPU 服务器往往会有多张 GPU 卡，为了确保不同的用户 GPU 任务之间不相互影响，我们可以设定我们运行的任务只使用其中某几张卡。

In [1]:
import os
import torch

# 设定当前进程只使用 0,1,2,3 四张卡
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

num_gpus = torch.cuda.device_count()

print("num_gpus:", num_gpus)

num_gpus: 4


## 引言

`torch.distributed` 是 PyTorch 提供的分布式计算工具包，支持大规模分布式训练和模型并行化。它允许在多个设备（如 GPU）和节点上并行训练深度学习模型，是实现高效分布式训练的关键工具之一。
`torch.distributed`中支持的分布式相关特性可以归纳为三个方面的组件：

### [分布式数据并行训练](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html) (DDP)

DDP 是一种广泛采用的单程序多数据（SPMD）训练范式。在 DDP 中，模型会在每个进程中进行复制，每个模型副本将处理不同的数据样本集。DDP 负责在进程间进行梯度通信，确保模型副本同步，并通过将梯度计算与梯度通信重叠来加速训练。DDP 的异步处理机制有效隐藏了通信的开销。

### [基于 RPC 的分布式训练](https://pytorch.org/docs/stable/rpc.html) (RPC)

RPC 支持无法适配数据并行训练的通用训练结构，例如分布式流水线并行、参数服务器模式，以及 DDP 与其他训练范式的组合。它能够管理远程对象的生命周期，并扩展了 [autograd 引擎](https://pytorch.org/docs/stable/autograd.html) 的功能，使其能够跨越机器边界。

RPC 在 DDP 之外提供了一种更通用的分布式方案，例如分布式流水线并行和参数服务器模式等。

### [集合通信](https://pytorch.org/docs/stable/distributed.html) (c10d)

该库支持在一个组内的进程之间发送张量。它提供了集合通信 API（如 `all_reduce` 和 `all_gather`）以及点对点（P2P）通信 API（如 `send` 和 `isend`）。DDP 和 RPC ([ProcessGroup Backend](https://pytorch.org/docs/stable/rpc.html#process-group-backend)) 都是基于 c10d 实现的，其中 DDP 使用集合通信，而 RPC 使用 P2P 通信。

通常情况下，开发者不需要直接使用这些底层通信 API，因为 DDP 和 RPC API 已能满足许多分布式训练场景。然而，在某些情况下，这些 API 依然非常有用。例如，在分布式参数平均的场景中，应用希望在反向传播之后计算所有模型参数的平均值，而不是通过 DDP 进行梯度通信。这样可以将通信与计算解耦，从而对通信内容实现更细粒度的控制，但同时也放弃了 DDP 提供的性能优化。[使用 PyTorch 编写分布式应用](https://pytorch.org/tutorials/intermediate/dist_tuto.html) 展示了如何使用 c10d 通信 API 的示例。

c10d 提供了底层的集合通信和点对点通信 API，为用户提供了更灵活的接口，以满足一些特定的需求。

## 数据并行训练

PyTorch 提供了多种数据并行训练的选项。对于从简单到复杂、从原型到生产逐步发展的应用程序，通常的开发路径如下：

* **单机单卡无分布式**：如果数据和模型可以放入一张 GPU 中，并且对训练速度没有太高要求，可以使用单设备训练。
* **单机多卡多线程分布式**：使用单机多 GPU 的 [DataParallel](https://pytorch.org/docs/stable/generated/torch.nn.DataParallel.html)，可以利用一台机器上的多张 GPU 来加速训练，同时仅需对代码进行最小的改动。
* **单机多卡多进程分布式**：如果需要进一步加速训练，并且愿意多写一些代码来进行设置，可以使用单机多 GPU 的 [DistributedDataParallel](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html)。
* **多机多卡多进程分布式**：如果应用需要跨机器扩展，可以使用多机的 DistributedDataParallel 和 [启动脚本](https://github.com/pytorch/examples/blob/master/distributed/ddp/README.md)。
* **有伸缩功能的多机分布式**：如果预期可能会出现错误（例如，内存不足）或在训练期间资源可能动态加入或离开，可以使用 [torch.distributed.elastic](https://pytorch.org/docs/stable/distributed.elastic.html) 启动分布式训练。

## `DataParallel`

单机多GPU下最简单的并行方案，只需要简单几行代码的修改。但是它的性能一般都不是最优的，一个是因为每次 `forward` 时，它都需要在多个卡上进行复制模型，另一个是因为 `DataParallel` 用的是多线程的试，所以它也会受到 Python GIL锁 的影响。

<div class="wy-nav-content-img">
    <img src="assets/Distributed_dp_procedure.png" width="800px" alt="DP 的训练步骤 ">
    <p>图1: DP 的训练步骤 </p>
</div>

相关教程：

* 模型介绍：[DataParallel](https://pytorch.org/docs/stable/generated/torch.nn.DataParallel.html)
* 教程：[Optional: Data Parallelism](https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html)

In [ ]:
import torch
import torch.nn as nn
from torch import optim

model = nn.Linear(5, 10)
devices = [torch.device("cuda", i) for i in range(num_gpus)]

########################################
if len(devices) > 1:
    model = nn.DataParallel(model, devices)
#########################################

model.to(devices[0])

loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

data = torch.randn(8, 5)
data = data.to(device=devices[0])
outputs = model(data)
print(outputs.shape)

labels = torch.randn_like(outputs)
loss_fn(outputs, labels).backward()
optimizer.step()

torch.Size([8, 10])


## `DistributedDataParallel`

相较于`DataParallel`，`DistributedDataParallel`会多需要几步设置，主要是用于调用`init_process_group`。另外 DDP 只需要在初始化构建的时候进行一次模型的Broadcast，而不是像 DP 一样，每次 forward 都需要进行模型拷贝。

DDP的相关参考资料为：

* [DDP notes](https://pytorch.org/docs/stable/notes/ddp.html): 通过一个简单的示例快速说明一个DDP的流程是什么样的，相比于单机单卡的代码的改动，同时也介绍了一些内部实现的原理。
* [Getting Started with Distributed Data Parallel](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html): 进一步说明了DDP的使用方法，并介绍了在DDP下如何保存了checkpoint，以及如何将DDP和MP进行结合，最后也引入了使用`torchrun`来对DDP进行初始化的方法。
* [Writing Distributed applications with pytorch](https://pytorch.org/tutorials/intermediate/dist_tuto.html): 教程中介绍了`torch.distributed`模块中的一些分布式通信的原语支持，包括了点对点的通信`send/recv`和`isend/irecv`，以及集合通信的几种模式（Scatter/Gather/Reduce/AllReduce/Broadcast/All-Gather），最后实现了一个简单的同步的分布式SGD的训练流程。
* [Launching and configuring distributed data parallel applications ](https://github.com/pytorch/examples/blob/main/distributed/ddp/README.md): 介绍了使用`torch.distributed.launch`来初始化DDP的方法，这个方法看起来已经完全被`torchrun`替代了，因为它内部实际调用的也是`torch.distributed.run`

`DistributedDataParallel`的一个简单的示例如下：

In [ ]:
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP
import os


def steup(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29544"
    dist.init_process_group("nccl", rank=rank, world_size=world_size)


def cleanup():
    dist.destroy_process_group()


def example(rank, world_size):
    steup(rank, world_size)

    model = nn.Linear(10, 10).to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    outputs = ddp_model(torch.randn(20, 10).to(rank))
    labels = torch.randn(20, 10).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()


def main():
    world_size = 2
    # 多进程代码不能直接在notebook中运行，需要直接用python解释器来运行
    # mp.spawn(example, args=(world_size,), nprocs=world_size, join=True)


if __name__ == "__main__":
    main()

如上面的示例代码所示：DDP 把底层的分布式通信全都隐藏起来了，当前我调用`backward()`时，梯度会一边进行反向计算，一边在多个结点之间进行同步通信。当`backward()`返回时，每个参数的梯度`param.grad`都已经包括了同步后的梯度。

### DDP 的训练流程

<div class="wy-nav-content-img">
    <img src="assets/Distributed_dpp_procedure.png" width="600px" alt="DDP 的训练步骤 ">
    <p>图2: DDP 的训练步骤 </p>
</div>

从上图中我们可以看出，DDP 的大致训练步骤如下：

1. 使用多个进程，每个进程都加载数据与模型
2. 各个进程同时进行前向传播，得到输出
3. 各个进程分别计算 loss，反向传播，计算梯度
4. 当在 backward 时，计算得到各个参数梯度后，会出发 hook，在各个进程间进行通信，将梯度在各个进程的 GPU 上进行同步
5. 各个进程分别更新模型。

### `rank`、`world_size`、`deivce`

通过上面代码可以看出，每个进程都需要感知的变量有：

* `rank`表示当前进程在分布式环境中的唯一标识符。每个进程都有一个独特的`rank`值，用于区分不同的进程。`rank`的取值范围是从 `0` 到 `world_size - 1`。
* `world_size` 表示分布式环境中的进程总数。它表示了整个分布式训练的规模，即参与训练的进程数量。
* `device` 表示本进程在结点上使用的 gpu 的设备 id

在单机情况下，我可以用`rank`来计算`device`，但在多结点下，我们必须计算好对应的`device`，可以通过 `rank % gpus_per_node` 来计算出来。我们可以通过下图来理解多机情况下，一些环境变量的意义。

<div class="wy-nav-content-img">
    <img src="assets/Distributed_ddp_multi-node_topo.png" width="800px" alt="DDP 在多机情况下的拓扑以及对应的各个环境变量的意义">
    <p>图3: DDP 在多机情况下的拓扑以及对应的各个环境变量的意义 </p>
</div>

### Skewed(偏斜) Processing Speeds

在DDP的每个进程时，DDP的构造、Forward、Backward都是分布式的同步点，原则上要求不同进程能够在差不多的时间到达同步点，如果有一些进程处理的过快或过慢，变会导致存在进程在同步点的位置长时间阻塞，时间过长的话，则会触发timeout，所以这就需要用户平衡好不同的结点，不同卡的计算负载。

但有间进程不同步不可避免，因为一些网络原因，资源限制或其他意外异常，所以在调用`init_process_group`时，需要设置好一些合理的timeout值。

Skewed 的问题一般在多 GPU 并行训练时，存在不同的 GPU 型号时会报一些 Warning。

### DDP模式下如何保存和加载Checkpoint

为了避免所有进程同时保存了模型快照，我们可以只让 `rank=0` 的进程来保存快照，加载时所有进程都从快照加载模型。

保存Checkpoint的代码如下：

```python
CHECKPOINT_PATH = tempfile.gettempdir() + "/model.checkpoint"
if dist.get_rank() == 0:
    # All processes should see same parameters as they all start from same
    # random parameters and gradients are synchronized in backward passes.
    # Therefore, saving it in one process is sufficient.
    torch.save(ddp_model.state_dict(), CHECKPOINT_PATH)

# Use a barrier() to make sure that process 1 loads the model after process
# saves it.
dist.barrier()
```

加载Checkpoint的代码如下：

```python
# configure map_location properly
# 这里假设是每个rank使用对应的device id
map_location = {'cuda:%d' % 0: 'cuda:%d' % rank}
state_dict = torch.load(CHECKPOINT_PATH, map_location=map_location)
ddp_model.load_state_dict(state_dict)
```

### DDP 的内部原理


* **Prerequisite**: 准备阶段。DDP 依赖 `c10d` 的进程组来进行通信，所以在实例化构造 DDP 之前，需要先初始化一个进程组，也就是需要调用`init_process_group`
* **Construction**: 在 DDP 的构造阶段。
    * DDP 的实例从 一个 local 的 torch Module 开始构建，但是会立即在 rank=0 的进程里 `boardcasts` 参数 `state_dict()`，从而保证在初始阶段，所有进程上拿到的模型参数完全一致。
    * 每个 DDP 进程都会创建一个`Reducer`，它来负责后续梯度的同步工作，`Reducer` 将整个模型的参数按 `bucket` 来管理，每一次进行分布式 reduce 操作都会在一个`bucket`上进行。
    * `Reducer`会向`autograd`注册hooks，每个参数一个 hook，这些 hooks 会在 bacward pass 时被触发。
* **Forward Pass**: 在前向过程中，如果`find_unused_parameters`被设置为True，那么 DDP 会对计算图进行遍历分析，实际计算的可能只是一个子图，对于不需要计算的参数，DDP会将其标记为 ready for reduce
* **Backward Pass**: 由于`backward()`方法是直接在 Loss Tensor 上执行的，这个已经脱离了`DDP`的上下文了。
    * 当一个参数对应梯度计算出来后，就会触 发 DDP hook 来标记这个参数梯度为 ready for reduction
    * 当有一个 bucket 中的所有梯度都是 ready 状态时，Reducer 就会执行 `allreduce` 操作来计算所有进程的平均梯度。
    * 所有进程中的 `Reducer` 会按相同的 bucket 的顺序来执行 `allreduce` ，而不是按照这些 bucket 的 ready 顺序。
* **Optimizer Step**: 从优化器的角度看，它看到的就只是一个本地的模型，没有任何区别。

<div class="wy-nav-content-img">
    <img src="assets/Distributed_ddp_reducer.png" width="600px" alt="DDP 中参数的梯度按 bucket 组织进行分组执行 AllReduce 操作">
    <p>图4: DDP 中参数的梯度按 bucket 组织进行分组执行 AllReduce 操作</p>
</div>

以下的代码示例演示了，通过`autograd`中的注册`hook`的机制，在`backward`时执行一些自定义的操作。

In [ ]:
x = torch.tensor([2.0, 3.0], requires_grad=True)


def custom_hook(grad):
    print(f"Original gradient: {grad}")
    # 修改梯度，例如将梯度乘以常数 0.5
    modified_grad = grad * 0.5
    print(f"Modified gradient: {modified_grad}")
    return modified_grad


hook_handle = x.register_hook(custom_hook)

y = x.pow(2).sum()
# 前向传播
print(f"Forward result: {y.item()}")

# 反向传播
y.backward()

# 打印修改后的梯度
print(f"Gradient after backward: {x.grad}")

# 移除 Hook（可选）
hook_handle.remove()

Forward result: 13.0
Original gradient: tensor([4., 6.])
Modified gradient: tensor([2., 3.])
Gradient after backward: tensor([2., 3.])


### DDP 模式下的数据加载

当我们使用 DDP 时，本质上还是一种数据并行模式，不同的进和需要加载各自负责的不同的进程，所以我们需要对数据的加载有区别于单机单卡的加载模式，在 Pytorch 中，提供了 `DistributedSampler` 方便我们在 DDP 环境下进行数据加载。

In [ ]:
from torch.utils.data import DistributedSampler, DataLoader
from torchvision import datasets

train_data = datasets.CIFAR10(root="../data", train=True, download=True)
# dist_sampler = DistributedSampler(train_data, num_replicas=1, drop_last=True)
# train_loader = DataLoader(train_data, batch_size=32, sampler=dist_sampler)

Files already downloaded and verified


虽然 `DistributedSampler` 可以解决多个进程加载数据的问题，但由于用户可能设置不同的 `world_size`，所以会出现数据不能被 `world_size`整除的问题，会有一个进程读取到的数据小于 `batch_size`。这种情况下，可以有多种处理方案：

1. 使用 `DistributedSampler` 的 `drop_last`参数，直接将多余的数据丢弃掉。
2. 手动对数据做一些 Padding。
3. 自定义采样器，当数据不足时，从已经读取的数据中再采样一些数据补充进来。

## `DP` 和 `DDP` 的区别

1. `DataParallel`是单进程多线程的方式，所以这就限定了`DataParallel`只能在单结点下使用，而`DistributedDataParallel`使用的是多进程模式，可以同时用于单结点内或多结点间。一般来说`DataParallel`是比`DistributedDataParallel`慢的，因为`DataParallel`会受限于GIL锁，同时因为每次迭代时，都需要进行模型复制、输入 scattering 和输出 gathering 。
2. `DistributedDataParallel`可以很好的和一些模型并行的分布式方案一起使用，每个 DDP 进程内使用多卡上的模型并行，而`DataParallel`则不支持。

## `torch.distributed.elastic`

`torchrun`

随着模型与数据规模的增加，分式下单点的容错变得十分必要，因为随机分布式结点的变多，不可避免的会出现一些进程遇到像OOM或IO异常的问题。而这样的单点失败的问题，在DDP里是解决不了的，因为DDP要求进程组里的所有进程要保持几乎完全的同步，一旦有一个进程挂了，那么基本进程基本都会卡死在 `AllReduce` 上。

`torch.distributed.elastic`分布式模块增加了错误容忍，从而使得在分布式训练时，可以使用一个动态的 pool

模块文档链接：https://pytorch.org/docs/stable/distributed.elastic.html

知乎上的Pytorch弹性训练原理解析：https://zhuanlan.zhihu.com/p/519410235

## RPC-Based Distributed Traning

基于RPC的分布式训练旨在提供一种更加通用的分布式机制，可以应用于一些不适合使用数据并行（data parallelism）的场景，例如：参数服务器和流水线并行等模式，以及多个观察者（observers）和代理（agents）的强化学习等应用。

`torch.distributed.rpc`由以下四个主要模块组成：

- [RPC](https://pytorch.org/docs/stable/rpc.html#rpc)：支持在远程工作节点上运行指定的函数。
- [RRef](https://pytorch.org/docs/stable/rpc.html#rref)：帮助管理远程对象的生命周期。参考计数协议详见 [RRef说明](https://pytorch.org/docs/stable/rpc/rref.html#remote-reference-protocol)。
- [分布式 Autograd](https://pytorch.org/docs/stable/rpc.html#distributed-autograd-framework)：将自动微分引擎扩展到跨机器的环境。更多细节请参考 [分布式Autograd设计](https://pytorch.org/docs/stable/rpc/distributed_autograd.html#distributed-autograd-design)。
- [分布式 Optimizer](https://pytorch.org/docs/stable/rpc.html#module-torch.distributed.optim)：借助分布式Autograd引擎计算的梯度，自动与所有参与的工作节点通信以更新参数。

主要教程如下：

1. [分布式RPC框架入门](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html)教程：通过一个简单的强化学习（RL）示例演示了`RPC`和`RRef`的使用。随后，它将基本的分布式模型并行应用于RNN示例，展示如何使用分布式 Autograd 和分布式 Optimizer。
2. [使用分布式RPC框架实现参数服务器](https://pytorch.org/tutorials/intermediate/rpc_param_server_tutorial.html)教程：借鉴了HogWild!训练的思想，将其应用于异步参数服务器（PS）训练。
3. [分布式流水线并行](https://pytorch.org/tutorials/intermediate/dist_pipeline_parallel_tutorial.html)教程：将单机流水线并行示例（参见单机模型并行最佳实践）扩展到分布式环境，并展示如何使用RPC实现。
4. [使用异步执行实现批量RPC处理](https://pytorch.org/tutorials/intermediate/rpc_async_execution.html)教程：展示如何使用`@rpc.functions.async_execution`装饰器实现RPC批量处理，这可以加速推理和训练。该教程使用了类似于教程1和2中的RL和PS示例。
5. [将分布式数据并行与分布式RPC框架结合](https://pytorch.org/tutorials/advanced/rpc_ddp_tutorial.html)教程：展示如何结合DDP和RPC，利用分布式数据并行和分布式模型并行进行模型训练。

## Communication Primitives

https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/DL2/High-performant_DL/Multi_GPU/hpdlmultigpu.html

<div class="wy-nav-content-img">
    <img src="assets/Distributed_collective_communication.drawio.svg" width="1000px" alt="Pytorch 中支持的一些集合通信原语">
    <p>图5: Pytorch 中支持的一些集合通信原语</p>
</div>

In [ ]:
import torch
import torch.distributed as dist
import os


def all_reduce(rank):
    t = torch.ones((5, 5), device=rank) * rank
    # t = p0.t + p1.t + p2.t + p3.t
    dist.all_reduce(t, op=dist.ReduceOp.SUM)
    assert t.mean().item() == 6  # [0, 1, 2, 3]


def reduce(rank):
    t = torch.ones((5, 5), device=rank) * rank
    dist.reduce(t, dst=0, op=dist.ReduceOp.SUM)
    # print(f"{os.getpid()}: {t.mean().item()}")
    if rank == 0:
        assert t.mean().item() == 6
    else:
        # 在gloo中结果不对
        assert t.mean().item() == rank


def boardcast(rank):
    t = torch.ones((5, 5), device=rank) * rank
    # 将rank 3的进程中的 t 广播到其他进程中
    dist.broadcast(t, src=3)
    assert t.mean().item() == 3


def all_gather(rank):
    t = torch.ones((5, 5), device=rank) * rank
    outputs = []
    for _ in range(dist.get_world_size()):
        outputs.append(torch.zeros((5, 5), device=rank))
    dist.all_gather(outputs, t)
    gather = torch.concat(outputs, dim=0)
    assert gather.shape == torch.Size([20, 5])
    assert gather.float().mean() == torch.tensor([0, 1.0, 2.0, 3.0]).mean()


def reduce_scatter(rank):
    world_size = dist.get_world_size()
    t = torch.ones((world_size * 5, 5), device=rank) * rank
    l = torch.split(t, 5, dim=0)
    reduce_rst = torch.zeros((5, 5), device=rank)
    dist.reduce_scatter(reduce_rst, list(l), dist.ReduceOp.SUM)
    assert reduce_rst.mean().item() == 6


def main_process(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "25321"
    os.environ["WORLD_SIZE"] = str(world_size)
    os.environ["RANK"] = str(rank)
    dist.init_process_group(backend="nccl")
    all_reduce(rank)
    reduce(rank)
    boardcast(rank)
    all_gather(rank)
    reduce_scatter(rank)
    dist.destroy_process_group()


if __name__ == "__main__":
    nprocs = 4
    # Not support in Notebook
    # mp.spawn(main_process, nprocs=nprocs, args=(nprocs,), join=True)